<div class="alert" style="background-color:#fff; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:15px 15px; color:#5d3a8e; font-size:40px'> 3.0 Feature Extraction - Combination of Features </h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> Table of Content</h2>
</div>

* [Required Libraries and Modules](#Required-Libraries-and-Modules)
* [Read Feature Pickle Files](#Read-Feature-Pickle-Files)
* [Features Grouping](#Features-Grouping)
* [Combination of Features](#Combination-of-Features)

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Required Libraries and Modules</h2>
</div>


In [1]:
import glob
import pickle
from scipy import sparse
from scipy.sparse import hstack
import numpy as np

import itertools
from scipy import sparse
from scipy.sparse import hstack
from tqdm._tqdm_notebook import tqdm_notebook
from tqdm import tqdm
tqdm_notebook.pandas()

C:\Users\Acer\AppData\Local\Temp/ipykernel_3752/661178399.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Read Feature Pickle Files</h2>
</div>


In [3]:
task='bully_binary_classification'
df = 'bully_data_clean_with_stopword'
path = task + "\\" + df + '\\features\\selected\\'
all_files = glob.glob(path + "/X*.pkl")
feature_dict = {}

for file_ in all_files:
    # temp = file_.split('\\')[-1].split('.')[0]
    temp = file_.split('\\')[-1].split('.')[0] # e.g X_AllTextual.pkl

    with open(file_,'rb') as f:
        x = pickle.load(f)
        feature_dict[temp]  = x

In [4]:
list(feature_dict.items())

[('X_CountVecWordCharAllTextStatSentimentAllDistilBertEmbeddingPycholinguisticLIWC22EmpathTermListsRatioToxicity',
  <112247x623045 sparse matrix of type '<class 'numpy.float64'>'
  	with 104362358 stored elements in Compressed Sparse Row format>),
 ('X_CountVecWordCharAllTextStatSentimentAllDistilBertEmbeddingPycholinguisticLIWC22EmpathToxicity',
  <112247x623039 sparse matrix of type '<class 'numpy.float64'>'
  	with 104213401 stored elements in Compressed Sparse Row format>),
 ('X_MBTI',
  <112247x16 sparse matrix of type '<class 'numpy.float64'>'
  	with 1795952 stored elements in Compressed Sparse Row format>)]

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Features Grouping</h2>
</div>


In [5]:
textual_group = ['X_CountVecWordCharAllTextStat']
                

sentiment_group = ['X_SentimentAll']

embedding_group = ['X_DistilBertEmbedding']
                  

# ['X_AlbertEmbedding'
# 'X_BertEmbedding',
# 'X_ElectraEmbedding',
# 'X_ggeluBertEmbedding',
# 'X_elmoEmbedding',
# 'X_nnlmEmbedding',
# 'X_FastTextEmbedding',
# 'X_GloveEmbedding',
# 'X_Word2VecEmbedding']

pycholinguistic_group = ['X_PycholinguisticLIWC22Empath']

termlist_group = ['X_TermListsRatio']

toxic_group = ['X_Toxicity']

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'>Combination of Features</h2>
</div>


In [ ]:
# Initialization
feature_comb_dict = {}
result = []

# Get Combination of feature from each group
for i in range(7):
    for subset in itertools.combinations([textual_group, 
                                         sentiment_group
                                         embedding_group, 
                                         pycholinguistic_group,
                                         termlist_group,
                                         toxic_group
                                         ], i):
        
        # Combination Feature
        result+= list(itertools.product(*subset))


# Individual Feature: No need since its been extracted
feature_comb_list = [i for i in result if len(i)>1]
print("Number of Features for combination: ",len(feature_comb_list))

# Loop over the code
for comb_set in tqdm(feature_comb_list):
    print('Combining features for: ',comb_set)
    for index, element in enumerate(comb_set):
        
        if index == 0:
            vector = feature_dict[element]
            vector_name = 'X_'+element.split('_')[-1]
        else:
            vector = sparse.hstack((vector, feature_dict[element]),format='csr')
            vector_name = vector_name+element.split('_')[-1]
 
    feature_comb_dict[vector_name] = vector
    print('Done Combining features for: ',comb_set)
    print()
    del vector

#         if len(subset) > 1:
#             print('Combining features for: ',subset)
#             for index, element in tqdm(enumerate(subset)):
#                 if index == 0:
#                     vector = feature_dict[element]
#                     vector_name = 'X_'+element.split('_')[-1]
#                 else:
#                     vector = hstack((vector, feature_dict[element]),format='csr')
#                     vector_name = vector_name+element.split('_')[-1]
                    
#             feature_comb_dict[vector_name] = vector


In [ ]:
##########################
# Output as pickle files #
##########################
data_set = df
for fname, fset in tqdm(feature_comb_dict.items()):
    with open(task+"\\"+data_set+"\\features\\selected\\new\\comb\\"+ fname + ".pkl",'wb') as f:
        pickle.dump(fset, f)

In [ ]:
#######################################
# Check Output as pickle files length #
#######################################
for fname, fset in tqdm(feature_comb_dict.items()):
    print(fname)
    print(fset.shape)
    print()